In [1]:
from pathlib import Path
import pandas as pd

artifacts = Path("/media/cumulus/curation_data/modality_classifiers/models/cord19/")
data_path = Path("/media/cumulus/curation_data/modality_classifiers/data/cord19")

classifiers = {
    "classifier": "higher-modality",
    "classname": "",
    "path": artifacts / "higher-modality/resnet18_higher-modality_1.pt",
    "children": [
        {
            "classifier": "experimental",
            "classname": "exp",
            "path": artifacts / "experimental/resnet18_experimental_2.pt",
            "children": [],
        },
        {
            "classifier": "graphics",
            "classname": "gra",
            "path": artifacts / "graphics/resnet18_graphics_1.pt",
            "children": [],
        },
        {
            "classifier": "microscopy",
            "classname": "mic",
            "path": artifacts / "microscopy/resnet18_microscopy_2.pt",
            "children": [
                # {
                #     "classifier": "electron",
                #     "classname": "mic.ele",
                #     "path": artifacts / "microscopy/electron_1.pt",
                #     "classes": ["mic.ele.sca", "mic.ele.tra"],
                #     "children": [],
                # },
            ],
        },
        {
            "classifier": "molecular",
            "classname": "mol",
            "path": artifacts / "molecular" / "resnet18_molecular_1.pt",
            "children": [],
        },
        {
            "classifier": "radiology",
            "classname": "rad",
            "path": artifacts / "radiology" / "resnet18_radiology_1.pt",
            "children": [],
        },
    ],
}



model_path = data_path / "cord19_microscopy_v1.parquet"
df = pd.read_parquet(model_path)
# df = df[:10]

In [2]:
from image_modalities_classifier.models.predict import ModalityPredictor, RunConfig
from os import cpu_count

config = RunConfig(32, cpu_count(), "cuda:0")
predictor = ModalityPredictor(classifiers, config)

In [3]:
base_img_path = "/media/cumulus/curation_data"
img_paths = df.img_path.values.tolist()
predictions = predictor.predict(img_paths, base_img_path)


(21547, 2)
['mic' 'mic' 'mic' ... 'mic' 'mic' 'mic']
exp
(277, 2)
['exp.pla' 'exp.pla' 'exp.pla' 'exp.pla' 'exp.pla' 'exp.gel' 'exp.pla'
 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel'
 'exp.gel' 'exp.gel' 'exp.gel' 'exp.pla' 'exp.pla' 'exp.pla' 'exp.gel'
 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.pla' 'exp.gel' 'exp.gel'
 'exp.gel' 'exp.gel' 'exp.pla' 'exp.pla' 'exp.gel' 'exp.gel' 'exp.gel'
 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.pla' 'exp.pla'
 'exp.pla' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.pla'
 'exp.gel' 'exp.pla' 'exp.pla' 'exp.pla' 'exp.gel' 'exp.gel' 'exp.gel'
 'exp.gel' 'exp.gel' 'exp.gel' 'exp.pla' 'exp.gel' 'exp.gel' 'exp.gel'
 'exp.gel' 'exp.gel' 'exp.pla' 'exp.gel' 'exp.gel' 'exp.gel' 'exp.gel'
 'exp.gel' 'exp.gel' 'exp.pla' 'exp.pla' 'exp.gel' 'exp.gel' 'exp.gel'
 'exp.pla' 'exp.gel' 'exp.pla' 'exp.gel' 'exp.pla' 'exp.pla' 'exp.pla'
 'exp.pla' 'exp.pla' 'exp.pla' 'exp.pla' 'exp.gel' 'exp.gel' 'exp.pla'
 'exp.pla'

In [4]:
predictions[:100]

,img_path,prediction
0,subfigure-classification/2016/train/DMFL/11373...,mic.flu
1,subfigure-classification/2016/train/DMFL/11373...,mic.flu
2,subfigure-classification/2016/train/DMFL/11373...,mic.lig
3,subfigure-classification/2016/train/DMFL/11373...,mic.flu
4,subfigure-classification/2016/train/DMFL/11373...,mic.flu
...,...,...
95,subfigure-classification/2016/train/DMFL/1471-...,mic.flu
96,subfigure-classification/2016/train/DMFL/1471-...,mic.flu
97,subfigure-classification/2016/train/DMFL/1471-...,mic.flu
98,subfigure-classification/2016/train/DMFL/1471-...,mic.flu


In [5]:
predictions.prediction.unique()

array(['mic.flu', 'mic.lig', 'exp.pla', 'mic.ele', 'pho', 'oth',
       'rad.xra', 'exp.gel', 'rad.ang', 'gra.oth', 'rad.cmp', 'rad.uls'],
      dtype=object)